In [ ]:
import requests

apikey=''

def get_lat_lng(address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={apikey}"
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        result = data['results'][0]
        lat = result['geometry']['location']['lat']
        lng = result['geometry']['location']['lng']
        return lat, lng
    else:
        return None

# Example usage
address = "1600 Amphitheatre Parkway, Mountain View, CA"
lat, lng = get_lat_lng(address)
print(f"Latitude: {lat}, Longitude: {lng}")